In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 55.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=90f7e33781f081cc8a7dac59c0315644ec1b0a8f131a75058076563f2672eb6d
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark 

from pyspark import SparkContext, SparkConf 

sc = SparkContext("local","spark_code") 
 

data = [['A', 'History', 'Male', 84], ['D', 'Chemistry', 'Male', 63], ['E', 'History', 'Male', 99], ['B', 'Chemistry', 'male', 19], ['C', 'History', 'Female', '94'],  ['C', 'Maths', 'Female', 95], ['A', 'Maths', 'Male', 75], ['C', 'Chemistry', 'Female', 92],  ['D', 'History', 'Male', 72], ['A', 'CS', 'Male', 57], ['A', 'Biology', 'Male', 92],  ['B', 'Maths', 'Female', 92], ['D', 'Biology', 'Male', 71], ['E', 'Maths', 'Male', 82], ['E', 'CS', 'Male', 84], ['B', 'CS', 'Female', 37],  ['B', 'Biology', 'Female', 38],  ['C', 'CS', 'Female',73], ['A', 'Chemistry', 'Male', 94],  ['B', 'History', 'Female', 89], ['C', 'Biology', 'Female', 93], ['D', 'Maths', 'Male', 83], ['E', 'Chemistry', 'Male', 99], ['D', 'CS', 'Male', 74], ['E', 'Biology', 'Male', 100]] 

In [ ]:
rdd = sc.parallelize(data)
rdd = rdd.map(lambda x: [x[1],(int(x[3]),1)])
rdd = rdd.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1])).map(lambda x: [x[0],x[1][0]/x[1][1]])

In [ ]:
rdd.collect()

[['History', 87.6],
 ['Chemistry', 73.4],
 ['Maths', 85.4],
 ['CS', 65.0],
 ['Biology', 78.8]]

In [ ]:
rdd = rdd.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])).map(lambda x: [x[0],x[1][0]/x[1][1]])
rdd = rdd.sortBy(lambda a: -a[1])

In [ ]:
rdd.collect()

[['Male', 87.66666666666667], ['Female', 65.5]]

In [ ]:
rdd = rdd.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])).map(lambda x: [x[0],x[1][0]/x[1][1]])

[['Male', 87.66666666666667], ['Female', 65.5]]

In [ ]:
rdd = rdd.sortBy(lambda a: a[1])

[['Female', 65.5], ['Male', 87.66666666666667]]

In [ ]:
import sys
import re
from pyspark import SparkContext, SparkConf
import numpy as np

import time as time
from operator import itemgetter

In [ ]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")

In [ ]:
filepath = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/email-Eu-core.txt'
links = sc.textFile(filepath)

#links.repartition(32)

#transpose
links_formatted = links.map(lambda x: (int(x.split(' ')[0]),int(x.split(' ')[1])))
links_formatted_t = links.map(lambda x: (int(x.split(' ')[1]),int(x.split(' ')[0])))


In [ ]:
links_formatted_list = links_formatted.map(lambda x: (x[0],[x[1]]))
link_formatted_dict = links_formatted_list.reduceByKey(lambda a,b: a+b)

#transpose
links_formatted_list_t = links_formatted_t.map(lambda x: (x[0],[x[1]]))
link_formatted_dict_t = links_formatted_list_t.reduceByKey(lambda a,b: a+b)

print('The number of nodes in L is ',link_formatted_dict.count())
print('The number of nodes in L.T is ',link_formatted_dict_t.count())


The number of nodes in L is  868
The number of nodes in L.T is  991


In [ ]:
hubs = links_formatted.flatMap(lambda x: x).distinct().map(lambda x: (x, 1.0))
n_nodes = hubs.count()

print('Total number of nodes is ', n_nodes)
elements = list(np.zeros(n_nodes))
rdd = sc.parallelize(elements)

rdd = rdd.zipWithIndex()

def getList(x):
  return_list = list(np.zeros(n_nodes))
  return return_list

matRdd = rdd.map(lambda x: getList(x[0]))

#transpose
matRdd_t = rdd.map(lambda x: getList(x[0]))


Total number of nodes is  1005


In [ ]:
def getList_empty(x):
  return_list = []
  return return_list

empty_rdd = rdd.map(lambda x: getList_empty(x[0]))
empty_rdd = empty_rdd.zipWithIndex()
empty_rdd = empty_rdd.map(lambda x: (x[1], x[0]))

def add_two_lists(x):
  if (x[1][0]) == None :
    return (x[0],['A'])
  else:
    return (x[0],x[1][0])

zip_empty = link_formatted_dict.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p = zip_empty.repartition(1)

#transpose
zip_empty_t = link_formatted_dict_t.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p_t = zip_empty_t.repartition(1)


In [ ]:
matRdd1 = matRdd.zipWithIndex()
matRdd_zip = matRdd1.map(lambda x: (x[1], x[0]))

#transpose
matRdd_t1 = matRdd_t.zipWithIndex()
matRdd_t_zip = matRdd_t1.map(lambda x: (x[1], x[0]))

sampleWordsRDD6 = zip_empty_p.join(matRdd_zip)
print(sampleWordsRDD6.count())

#transpose
sampleWordsRDD6_t = zip_empty_p_t.join(matRdd_t_zip)
print(sampleWordsRDD6_t.count())


1005
1005


In [ ]:
def form_matrix(x):
  if x[1][0][0] == 'A':
    l1 = [0 for i in range(n_nodes)]
    return (x[0],l1)
  else:
    for k in x[1][0]:
      x[1][1][k] = 1
    return (x[0],x[1][1])

l_matrix = sampleWordsRDD6.map(lambda x: form_matrix(x))

#transpose
l_t_matrix = sampleWordsRDD6_t.map(lambda x: form_matrix(x))

In [ ]:
hubs_ones = [1 for i in range(n_nodes)]
hubs = sc.parallelize(hubs_ones)
hubs_updated = hubs_ones

def product_sum(x):
  if x[1][1] == None:
    return 0
  else:
    return np.sum(np.array(x[1])*np.array(x[2]))

start0 = time.time()
for i in range(40):
  start = time.time()

  matrix_hubs = l_matrix.map(lambda x: (x[0],x[1],hubs_updated))
  matrix_product_hubs = matrix_hubs.map(lambda x: product_sum(x))

  auths_list = matrix_product_hubs.collect()
  auths_max = max(auths_list)
  auths_list = [auths_list[i]/auths_max for i in range(len(auths_list))]

  matrix_t_auths = l_t_matrix.map(lambda x: (x[0],x[1],auths_list))
  matrix_t_product_auths = matrix_t_auths.map(lambda x: product_sum(x))

  hubs_updated = matrix_t_product_auths.collect()
  hub_max = max(hubs_updated)
  hubs_updated = [hubs_updated[i]/hub_max for i in range(len(hubs_updated))]

  print(i, np.round(time.time() - start,4))

print('Total time : ', np.round(time.time() - start0,4))

0 1.2942
1 1.3772
2 1.3551
3 1.3098
4 1.4123
5 1.4104
6 1.296
7 1.3519
8 1.3559
9 1.3616
10 1.2615
11 1.5256
12 1.4017
13 1.5182
14 1.3605
15 1.3615
16 1.555
17 1.4034
18 1.3499
19 1.2968
20 1.36
21 1.3647
22 1.3338
23 1.2766
24 1.3685
25 1.3707
26 1.397
27 1.3744
28 1.3521
29 1.312
30 1.3334
31 1.4049
32 1.416
33 1.2978
34 1.3425
35 1.3537
36 1.4556
37 1.3452
38 1.3469
39 1.3396
Total time :  54.7352


In [ ]:
tuple_hubs = []
tuple_auths = []

for i in range(0, n_nodes):
  t = (i, hubs_updated[i])
  tuple_hubs.append(t)

  t = (i, auths_list[i])
  tuple_auths.append(t)

sorted_hubs_asc = sorted(tuple_hubs,key=itemgetter(1), reverse=False)
sorted_hubs_dsc = sorted(tuple_hubs,key=itemgetter(1), reverse=True)

sorted_auths_asc = sorted(tuple_auths,key=itemgetter(1), reverse=False)
sorted_auths_dsc = sorted(tuple_auths,key=itemgetter(1), reverse=True)

In [ ]:
print('Top nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_dsc[i][0], np.round(sorted_hubs_dsc[i][1],4)))

Top nodes IDs with hub scores
1     182   1.0   
2     28    0.8942 
3     543   0.8555 
4     31    0.8101 
5     547   0.7635 


In [ ]:
print('Bottom nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_asc[i][0], np.round(sorted_hubs_asc[i][1],4)))

Bottom nodes IDs with hub scores
1     289   0.0   
2     407   0.0   
3     464   0.0   
4     488   0.0   
5     491   0.0   


In [ ]:
print('Top nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_dsc[i][0], np.round(sorted_auths_dsc[i][1],4)))

Top nodes IDs with authority scores
1     40    1.0   
2     17    0.7598 
3     656   0.7553 
4     99    0.7333 
5     629   0.6497 


In [ ]:
print('Bottom nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_asc[i][0], np.round(sorted_auths_asc[i][1],4)))

Bottom nodes IDs with authority scores
1     429   0.0   
2     430   0.0   
3     431   0.0   
4     432   0.0   
5     433   0.0   
